<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Initialization" data-toc-modified-id="Initialization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialization</a></span></li><li><span><a href="#Example-1:-Microchips-classification" data-toc-modified-id="Example-1:-Microchips-classification-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example 1: Microchips classification</a></span><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Plot-Data" data-toc-modified-id="Plot-Data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Plot Data</a></span></li></ul></li><li><span><a href="#Implementing-Logistic-Regression" data-toc-modified-id="Implementing-Logistic-Regression-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Implementing Logistic Regression</a></span></li><li><span><a href="#Run-Logistic-Regression" data-toc-modified-id="Run-Logistic-Regression-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Run Logistic Regression</a></span><ul class="toc-item"><li><span><a href="#Plot-Decision-Boundaries" data-toc-modified-id="Plot-Decision-Boundaries-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Plot Decision Boundaries</a></span></li><li><span><a href="#Prediction" data-toc-modified-id="Prediction-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Prediction</a></span></li></ul></li><li><span><a href="#Example-2-:-MNIST-Classification" data-toc-modified-id="Example-2-:-MNIST-Classification-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Example 2 : MNIST Classification</a></span><ul class="toc-item"><li><span><a href="#Training" data-toc-modified-id="Training-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Prediction" data-toc-modified-id="Prediction-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Prediction</a></span></li></ul></li></ul></div>

# Classification with Logistic Regression in Octave
This notebook demonstrates how to implement logistic regression in Octave.  
The examples are taken from Andrew Ng's [Stanford Machine Learning Course](https://www.coursera.org/learn/machine-learning)

## Initialization

In [ ]:
clear; close all; clc;

## Example 1: Microchips classification

### Load Data

In [ ]:
data = load('data/microchips_tests.csv');

X = data(:, 1:2);
y = data(:, 3);

### Plot Data

In [ ]:
% Find indices of positive and negative examples.
positiveIndices = find(y == 1);
negativeIndices = find(y == 0);

% Plot examples.
figure()
hold on;
plot(X(positiveIndices, 1), X(positiveIndices, 2), 'k+', 'LineWidth', 2, 'MarkerSize', 7);
plot(X(negativeIndices, 1), X(negativeIndices, 2), 'ko', 'MarkerFaceColor', 'y', 'MarkerSize', 7);

% Draw labels and Legend
xlabel('Microchip Test 1');
ylabel('Microchip Test 2');
legend('y = 1', 'y = 0');
hold off;

## Implementing Logistic Regression

In [ ]:
% SIGMOID function.
function g = sigmoid(z)
    g = 1 ./ (1 + exp(-z));
end

In [ ]:
% HYPOTHESIS function.
% It predicts the output values y based on the input values X and model parameters.
function [predictions] = hypothesis(X, theta)
    % Input:
    % X - input features - (m x n) matrix.
    % theta - our model parameters - (n x 1) vector.
    %
    % Output:
    % predictions - output values that a calculated based on model parameters - (m x 1) vector.
    %
    % Where:
    % m - number of training examples,
    % n - number of features.

    predictions = sigmoid(X * theta);
end

In [ ]:
% COST function.
% It shows how accurate our model is based on current model parameters.
function [cost] = cost_function(X, y, theta, lambda)
    % X - training set.
    % y - training output values.
    % theta - model parameters.
    % lambda - regularization parameter.

    % Initialize number of training examples.
    m = length(y); 

    % Calculate hypothesis.
    predictions = hypothesis(X, theta);

    % Calculate regularization parameter.
    % Remmber that we should not regularize the parameter theta_zero.
    theta_cut = theta(2:end, 1);
    regularization_param = (lambda / (2 * m)) * (theta_cut' * theta_cut);
    
    % Calculate cost function.
    cost = (-1 / m) * (y' * log(predictions) + (1 - y)' * log(1 - predictions)) + regularization_param;
end

In [ ]:
% GRADIENT STEP function.
% It performs one step of gradient descent for theta parameters.
function [gradients] = gradient_step(X, y, theta, lambda)
    % X - training set.
    % y - training output values.
    % theta - model parameters.
    % lambda - regularization parameter.

    % Initialize number of training examples.
    m = length(y); 

    % Initialize variables we need to return. 
    gradients = zeros(size(theta));

    % Calculate hypothesis.
    predictions = hypothesis(X, theta);

    % Calculate regularization parameter.
    regularization_param = (lambda / m) * theta;

    % Calculate gradient steps.
    gradients = (1 / m) * (X' * (predictions - y)) + regularization_param;
    
    % We should NOT regularize the parameter theta_zero.
    gradients(1) = (1 / m) * (X(:, 1)' * (predictions - y));
end

In [ ]:
% GRADIENT CALLBACK function.
% This function is used as a callback function for fminunc and it aggregates
% cost and gradient values.
function [cost, gradients] = gradient_callback(X, y, theta, lambda)
    % X - training set.
    % y - training output values.
    % theta - model parameters.
    % lambda - regularization parameter.

    % Calculate cost function.
    cost = cost_function(X, y, theta, lambda);

    % Do one gradient step.
    gradients = gradient_step(X, y, theta, lambda);
end

In [ ]:
% GRADIENT DESCENT function.
% Iteratively optimizes theta model parameters.
function [theta, J, exit_flag] = gradient_descent(X, y, theta, lambda, options)
    % X - training set.
    % y - training output values.
    % theta - model parameters.
    % lambda - regularization parameter.
    % options - fminunc options
  
    % Optimize
    [theta, J, exit_flag] = fminunc(@(t)(gradient_callback(X, y, t, lambda)), theta, options);
end


In [ ]:
% Output function to be passed to optimization algorithm fminunc
function stop = output_fcn(x, optimvalues, state)
  it = optimvalues.iter;
  if ( (it == 1) || (mod(it,10) == 0))
    fprintf("Iteration %d : %f\n", it, optimvalues.fval);
  endif
  stop = false;
end


In [ ]:
% LOGISTIC REGRESSION function.
% Calculate the optimal thetas for given training set and output values.
function [theta, J, J_history, exit_flag] = logistic_regression_train(X, y, lambda)
    % X - training set.
    % y - training output values.
    % lambda - regularization parameter.

    % Calculate the number of training examples.
    m = size(y, 1);

    % Calculate the number of features.
    n = size(X, 2);

    % Add a column of ones to X.
    X = [ones(m, 1), X];

    % Initialize model parameters.
    initial_theta = zeros(n + 1, 1);

    % Set options for fminunc
    options = optimset('GradObj', 'on', 'MaxIter', 400, 'OutputFcn', @output_fcn);
        
    % Run gradient descent.
    [theta, J, exit_flag] = gradient_descent(X, y, initial_theta, lambda, options);

    % Record the history of chaning J.
    J_history = zeros(1, 1);
    J_history(1) = cost_function(X, y, initial_theta, lambda);
    J_history(2) = cost_function(X, y, theta, lambda);
end


## Run Logistic Regression
We add more polynomial features in order to allow for a non-linear decision boundary.

In [ ]:
% Generates polynomial features of certain degree.
% This function is used to extend training set features with new features to get 
% more complex form/shape if decision boundaries.
function out = add_polynomial_features(X1, X2, degree)
    % Returns a new feature array with more features, comprising of 
    % X1, X2, X1.^2, X2.^2, X1*X2, X1*X2.^2, etc..
    out = ones(size(X1(:, 1)));
    for i = 1:degree
        for j = 0:i
            out(:, end + 1) = (X1 .^ (i - j)) .* (X2 .^ j);
        end
    end
end

In [ ]:
polynomial_degree = 2;
X_ext = add_polynomial_features(X(:, 1), X(:, 2), polynomial_degree);

In [ ]:
lambda = 1; % regularization parameter
[theta, J, J_history, exit_flag] = logistic_regression_train(X_ext, y, lambda);

fprintf('\n');
fprintf('Initial cost: %f\n', J_history(1));
fprintf('Optimized cost: %f\n', J);

### Plot Decision Boundaries

In [ ]:
% Generate a grid range.
u = linspace(-1, 1, 50);
v = linspace(-1, 1, 50);
z = zeros(length(u), length(v));
% Evaluate z = (x * theta) over the grid.
for i = 1:length(u)
    for j = 1:length(v)
        % Add polynomials.
        x = add_polynomial_features(u(i), v(j), polynomial_degree);
        % Add ones.
        x = [ones(size(x, 1), 1), x];
        z(i, j) = x * theta;
    end
end

% It is important to transpose z before calling the contour.
z = z';

figure()
hold on;
plot(X(positiveIndices, 1), X(positiveIndices, 2), 'k+', 'LineWidth', 2, 'MarkerSize', 7);
plot(X(negativeIndices, 1), X(negativeIndices, 2), 'ko', 'MarkerFaceColor', 'y', 'MarkerSize', 7);

% Plot z = 0
% Notice you need to specify the range [0, 0]
contour(u, v, z, [0, 0], 'LineWidth', 2);

% Draw labels and Legend
xlabel('Microchip Test 1');
ylabel('Microchip Test 2');
title(sprintf('lambda = %g', lambda));
legend('y = 1', 'y = 0', 'Decision boundary');
hold off;

### Prediction

In [ ]:
x = [
    0, 0;
    -0.5, -0.5
];

% Add polynomials.
x = add_polynomial_features(x(:, 1), x(:, 2), polynomial_degree);
% Add ones.
x = [ones(size(x, 1), 1), x];

probabilities = hypothesis(x, theta);

for example=1:length(probabilities)
   fprintf('%d : x=%0.1f \ty=%0.1f \tprob=%f \n', example, x(example, 3), x(example, 4), probabilities(example));
end

## Example 2 : MNIST Classification

In [ ]:
% Displays 2D data stored in X.
% Each raw of X is one squared image reshaped into vector.
function [h, display_array] = display_data(X)
  % Since every row in X is a squared image reshaped into vector we may calculate its width.
  example_width = round(sqrt(size(X, 2)));

  % Gray Image
  colormap(gray);

  % Compute rows, cols
  [m n] = size(X);
  example_height = (n / example_width);

  % Compute number of items to display
  display_rows = floor(sqrt(m));
  display_cols = ceil(m / display_rows);

  % Between images padding
  pad = 1;

  % Setup blank display
  display_array = -ones(pad + display_rows * (example_height + pad), pad + display_cols * (example_width + pad));

  % Copy each example into a patch on the display array
  curr_ex = 1;
  for j = 1:display_rows
    for i = 1:display_cols
      if curr_ex > m, 
        break; 
      end
      % Copy the patch
      
      % Get the max value of the patch
      max_val = max(abs(X(curr_ex, :)));

      row_shift = pad + (j - 1) * (example_height + pad) + (1:example_height);
      column_shift = pad + (i - 1) * (example_width + pad) + (1:example_width);

      display_array(row_shift, column_shift) = reshape(X(curr_ex, :), example_height, example_width) / max_val;
      curr_ex = curr_ex + 1;
    end
    if curr_ex > m, 
      break; 
    end
  end

  % Display Image
  h = imagesc(display_array, [-1 1]);

  % Do not show axis
  axis image off;

  drawnow;
end

In [ ]:
load('data/digits.mat');

% Plotting some training example ----------------------------------------------------
fprintf('Visualizing data...\n');

% Randomly select 100 data points to display
random_digits_indices = randperm(size(X, 1));
random_digits_indices = random_digits_indices(1:100);

figure(2);
display_data(X(random_digits_indices, :));

### Training

In [ ]:
% Trains multiple logistic regression classifiers and returns all
% the classifiers in a matrix all_theta, where the i-th row of all_theta 
% corresponds to the classifier for label i.
function [all_theta] = one_vs_all(X, y, num_labels, lambda, num_iterations)
    % Some useful variables
    m = size(X, 1);
    n = size(X, 2);

    % We need to return the following variables correctly 
    all_theta = zeros(num_labels, n + 1);

    % Add ones to the X data matrix.
    X = [ones(m, 1) X];

    for class_index=1:num_labels
        % Convert scalar y to vector with related bit being set to 1.
        y_vector = (y == class_index);

        % Set options for fminunc
        options = optimset('GradObj', 'on', 'MaxIter', num_iterations, 'OutputFcn', @output_fcn);

        % Set initial thetas to zeros.
        initial_theta = zeros(n + 1, 1);

        % Train the model for current class.
        [theta] = gradient_descent(X, y_vector, initial_theta, lambda, options);
        
        % Add theta for current class to the list of thetas.
        theta = theta';
        all_theta(class_index, :) = theta; 
    end
end

In [ ]:
% Setup the parameters you will use for this part of the exercise
input_layer_size = 400;  % 20x20 input images of digits.
num_labels = 10; % 10 labels, from 1 to 10 (note that we have mapped "0" to label 10).

fprintf('Training One-vs-All Logistic Regression...\n')
lambda = 0.01;
num_iterations = 50;
[all_theta] = one_vs_all(X, y, num_labels, lambda, num_iterations);

### Prediction

In [ ]:
% Predict the label for a trained one-vs-all classifier. The labels 
% are in the range 1..K, where K = size(all_theta, 1)
function p = one_vs_all_predict(all_theta, X)
    m = size(X, 1);
    num_labels = size(all_theta, 1);

    % We need to return the following variables correctly.
    p = zeros(size(X, 1), 1);

    % Add ones to the X data matrix
    X = [ones(m, 1) X];

    % Calculate probabilities of each number for each input example.
    % Each row relates to the input image and each column is a probability that this example is 1 or 2 or 3 etc. 
    h = sigmoid(X * all_theta');

    % Now let's find the highest predicted probability for each row.
    % Also find out the row index with highest probability since the index is the number we're trying to predict.
    [p_vals, p] = max(h, [], 2);
end

In [ ]:
fprintf('Predict for One-Vs-All...\n')
pred = one_vs_all_predict(all_theta, X);

fprintf('\nTraining Set Accuracy: %f\n', mean(double(pred == y)) * 100);